# Cleaning SMARD production and consumption datasets

In [1]:
import pandas as pd
import yaml

In [42]:
try:
    with open("./../config.yaml", 'r') as file:
        config = yaml.safe_load(file)
except Exception as e:
    print('Error reading the config file')

## Production

### Reading data

In [3]:
def read_data(file:str)->pd.DataFrame:
    return pd.read_csv(config["data"][file], delimiter=';')

In [4]:
prod = read_data('smard_production')
prod

,Date,Start,End,Biomass [MWh] Calculated resolutions,Hydropower [MWh] Calculated resolutions,Wind offshore [MWh] Calculated resolutions,Wind onshore [MWh] Calculated resolutions,Photovoltaics [MWh] Calculated resolutions,Other renewable [MWh] Calculated resolutions,Nuclear [MWh] Calculated resolutions,Lignite [MWh] Calculated resolutions,Hard coal [MWh] Calculated resolutions,Fossil gas [MWh] Calculated resolutions,Hydro pumped storage [MWh] Calculated resolutions,Other conventional [MWh] Calculated resolutions
0,"Jan 1, 2017",12:00 AM,12:00 AM,"40,328,389.5","15,590,070.5","17,414,191.5","85,190,050.75","35,883,181","1,754,386.25","72,213,976.5","129,286,596.75","66,005,509.5","25,580,368","9,290,780.25","48,743,064.75"
1,"Jan 1, 2018",12:00 AM,12:00 AM,"40,090,463","15,257,652.75","19,068,415.5","89,270,552.25","41,234,283","1,531,134.75","71,841,657","128,361,760.75","71,545,262.5","42,878,209.5","9,135,983.5","12,838,020.5"
2,"Jan 1, 2019",12:00 AM,12:00 AM,"40,387,606.5","15,830,486","24,382,827","99,878,949","41,914,770.25","1,464,378.75","71,042,164.25","102,728,678","47,815,284.5","54,619,705.25","9,020,582.75","12,730,557.75"
3,"Jan 1, 2020",12:00 AM,12:00 AM,"40,822,337.25","15,583,446.25","26,883,159.25","103,103,956.25","45,821,578","1,607,736.75","60,923,886.25","83,374,010","34,871,983.75","59,125,990.25","11,528,660","12,753,899.75"
4,"Jan 1, 2021",12:00 AM,12:00 AM,"39,460,166.75","14,457,191.25","24,010,320.5","89,407,873.5","46,606,884.75","1,542,812","65,405,986.5","98,202,233.25","51,841,756.25","52,405,148","8,721,439.25","12,991,351.5"
5,"Jan 1, 2022",12:00 AM,12:00 AM,"39,469,667.75","12,381,968.5","24,747,841","100,563,334.75","55,298,837.25","1,223,483.75","32,824,469.5","103,525,587","62,889,369.75","38,175,056","10,603,312.25","11,515,005.5"


In [5]:
prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 15 columns):
 #   Column                                             Non-Null Count  Dtype 
---  ------                                             --------------  ----- 
 0   Date                                               6 non-null      object
 1   Start                                              6 non-null      object
 2   End                                                6 non-null      object
 3   Biomass [MWh] Calculated resolutions               6 non-null      object
 4   Hydropower [MWh] Calculated resolutions            6 non-null      object
 5   Wind offshore [MWh] Calculated resolutions         6 non-null      object
 6   Wind onshore [MWh] Calculated resolutions          6 non-null      object
 7   Photovoltaics [MWh] Calculated resolutions         6 non-null      object
 8   Other renewable [MWh] Calculated resolutions       6 non-null      object
 9   Nuclear [MWh] Calculated 

### Changing dtype of values

In [6]:
def fix_dtypes(df:pd.DataFrame):

    df_temp = df.copy()

    #drop columns that are not needed
    df_temp.drop(['Start','End'], axis=1, inplace=True)

    #change date dtype
    df_temp['Date'] = pd.to_datetime(df_temp['Date'])

    
     #change value to float
    for col in df_temp.columns:
        if df_temp[col].dtype == 'object':
            df_temp[col] = df_temp[col].str.replace(',', '').astype('float')
   

    return df_temp
    

In [7]:
prod = fix_dtypes(prod)

### Quick inspection of totals

In [8]:
prod.set_index('Date').sum(axis=1)

Date
2017-01-01    5.472806e+08
2018-01-01    5.430534e+08
2019-01-01    5.218160e+08
2020-01-01    4.964006e+08
2021-01-01    5.050532e+08
2022-01-01    4.932179e+08
dtype: float64

These totals align with other data sources.

### Clean column names

In [9]:
def clean_col_names(df:pd.DataFrame)->pd.DataFrame:

    df_temp = df.copy()

    df_temp.columns = (df_temp.columns
                       .str.replace(' [MWh] Calculated resolutions', '')
                       .str.lower()
                       .str.replace(' ', '_')
                      )
    return df_temp

In [10]:
prod = clean_col_names(prod)
prod

,date,biomass,hydropower,wind_offshore,wind_onshore,photovoltaics,other_renewable,nuclear,lignite,hard_coal,fossil_gas,hydro_pumped_storage,other_conventional
0,2017-01-01,40328389.50,15590070.50,17414191.50,8.519005e+07,35883181.00,1754386.25,72213976.50,1.292866e+08,66005509.50,25580368.00,9290780.25,48743064.75
1,2018-01-01,40090463.00,15257652.75,19068415.50,8.927055e+07,41234283.00,1531134.75,71841657.00,1.283618e+08,71545262.50,42878209.50,9135983.50,12838020.50
2,2019-01-01,40387606.50,15830486.00,24382827.00,9.987895e+07,41914770.25,1464378.75,71042164.25,1.027287e+08,47815284.50,54619705.25,9020582.75,12730557.75
3,2020-01-01,40822337.25,15583446.25,26883159.25,1.031040e+08,45821578.00,1607736.75,60923886.25,8.337401e+07,34871983.75,59125990.25,11528660.00,12753899.75
4,2021-01-01,39460166.75,14457191.25,24010320.50,8.940787e+07,46606884.75,1542812.00,65405986.50,9.820223e+07,51841756.25,52405148.00,8721439.25,12991351.50
5,2022-01-01,39469667.75,12381968.50,24747841.00,1.005633e+08,55298837.25,1223483.75,32824469.50,1.035256e+08,62889369.75,38175056.00,10603312.25,11515005.50


### Get year from date

In [13]:
def get_year(df:pd.DataFrame)->pd.DataFrame:
    df_temp = df.copy()
    
    df_temp['year']=df_temp['date'].dt.year

    df_temp.drop('date', axis=1, inplace = True)

    # shift column 'Name' to first position
    first_column = df_temp.pop('year')
  
    # insert column using insert(position,column_name,
    # first_column) function
    df_temp.insert(0, 'year', first_column)

    return df_temp

In [14]:
prod = get_year(prod)

### Reshaping data

In [15]:
prod.head()

,year,biomass,hydropower,wind_offshore,wind_onshore,photovoltaics,other_renewable,nuclear,lignite,hard_coal,fossil_gas,hydro_pumped_storage,other_conventional
0,2017,40328389.50,15590070.50,17414191.50,8.519005e+07,35883181.00,1754386.25,72213976.50,1.292866e+08,66005509.50,25580368.00,9290780.25,48743064.75
1,2018,40090463.00,15257652.75,19068415.50,8.927055e+07,41234283.00,1531134.75,71841657.00,1.283618e+08,71545262.50,42878209.50,9135983.50,12838020.50
2,2019,40387606.50,15830486.00,24382827.00,9.987895e+07,41914770.25,1464378.75,71042164.25,1.027287e+08,47815284.50,54619705.25,9020582.75,12730557.75
3,2020,40822337.25,15583446.25,26883159.25,1.031040e+08,45821578.00,1607736.75,60923886.25,8.337401e+07,34871983.75,59125990.25,11528660.00,12753899.75
4,2021,39460166.75,14457191.25,24010320.50,8.940787e+07,46606884.75,1542812.00,65405986.50,9.820223e+07,51841756.25,52405148.00,8721439.25,12991351.50


In [16]:
prod.columns

Index(['year', 'biomass', 'hydropower', 'wind_offshore', 'wind_onshore',
       'photovoltaics', 'other_renewable', 'nuclear', 'lignite', 'hard_coal',
       'fossil_gas', 'hydro_pumped_storage', 'other_conventional'],
      dtype='object')

In [19]:
def reshape(df:pd.DataFrame)->pd.DataFrame:

    df_temp = df.copy()

    #get columns to use in reshape
    var_columns = df_temp.drop('year', axis=1).columns.tolist()

    #reshape from wide to long using pd.melt
    df_temp = (pd.melt(df_temp,
                       id_vars=['year'],
                       value_vars=var_columns,
                        var_name='source',
                        value_name = 'quantity_mwh')
              )
    
    return df_temp

In [26]:
prod = reshape(prod)

In [27]:
prod

,year,source,quantity_mwh
0,2017,biomass,40328389.50
1,2018,biomass,40090463.00
2,2019,biomass,40387606.50
3,2020,biomass,40822337.25
4,2021,biomass,39460166.75
...,...,...,...
67,2018,other_conventional,12838020.50
68,2019,other_conventional,12730557.75
69,2020,other_conventional,12753899.75
70,2021,other_conventional,12991351.50


### Changing units from MWh to TWh

In [34]:
def fix_units(df:pd.DataFrame, col: str)->pd.DataFrame:

    df_temp = df.copy()

    df_temp[col] = df_temp[col].apply(lambda x: x/1000000)
    
    df_temp = df_temp.rename(columns={col:'quantity_twh'})

    return df_temp

In [37]:
prod = fix_units(prod, 'quantity_mwh')

In [38]:
prod

,year,source,quantity_twh
0,2017,biomass,40.328390
1,2018,biomass,40.090463
2,2019,biomass,40.387606
3,2020,biomass,40.822337
4,2021,biomass,39.460167
...,...,...,...
67,2018,other_conventional,12.838021
68,2019,other_conventional,12.730558
69,2020,other_conventional,12.753900
70,2021,other_conventional,12.991352


## Consumption

### Reading data

In [44]:
consump = read_data('smard_consumption')
consump

,Date,Start,End,Total (grid load) [MWh] Calculated resolutions,Residual load [MWh] Calculated resolutions,Hydro pumped storage [MWh] Calculated resolutions
0,"Jan 1, 2017",12:00 AM,12:00 AM,"505,674,706.5","367,187,283.25","8,878,129"
1,"Jan 1, 2018",12:00 AM,12:00 AM,"509,156,603.75","359,583,353","12,471,072.5"
2,"Jan 1, 2019",12:00 AM,12:00 AM,"497,287,007.5","331,110,461.25","12,005,179.75"
3,"Jan 1, 2020",12:00 AM,12:00 AM,"485,290,404.25","309,481,710.75","13,744,805.75"
4,"Jan 1, 2021",12:00 AM,12:00 AM,"504,515,945.5","344,490,866.75","11,315,545.25"
5,"Jan 1, 2022",12:00 AM,12:00 AM,"482,637,633","302,027,620","14,350,349"


The residual load is the actual consumption or grid load minus the generation from photovoltaic installations and wind power stations. If the residual load is zero or negative, then renewable energy from wind or solar was able to meet the demand for energy. [SMARD.de](https://www.smard.de/en/204142-204142#:~:text=The%20forecasted%20residual%20load%20is,stations%20(onshore%20and%20offshore).)

### Changing dtypes of values

In [46]:
consump = fix_dtypes(consump)

### Quick inspection of totals

In [47]:
consump.set_index('Date').sum(axis=1)

Date
2017-01-01    8.817401e+08
2018-01-01    8.812110e+08
2019-01-01    8.404026e+08
2020-01-01    8.085169e+08
2021-01-01    8.603224e+08
2022-01-01    7.990156e+08
dtype: float64

### Clean column names

In [49]:
consump = clean_col_names(consump)

### Get year from date

In [51]:
consump = get_year(consump)

### Reshaping data

In [53]:
consump = reshape(consump)

### Changing units from MWh to TWh

In [57]:
consump = fix_units(consump, 'quantity_mwh')

In [58]:
consump

,year,source,quantity_twh
0,2017,total_(grid_load),505.674707
1,2018,total_(grid_load),509.156604
2,2019,total_(grid_load),497.287008
3,2020,total_(grid_load),485.290404
4,2021,total_(grid_load),504.515945
5,2022,total_(grid_load),482.637633
6,2017,residual_load,367.187283
7,2018,residual_load,359.583353
8,2019,residual_load,331.110461
9,2020,residual_load,309.481711
